<a href="https://colab.research.google.com/github/joaogabrielanuto/quant-traders/blob/under-construction/detector-topos-fundos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://tradingcomdados.com/images/logotipo/logotipo-trading-com-dados.svg" width="300" align="left"/>

---
# **Velocidade de cálculo das funções** 


#### Checagem do tempo gasto entre funções de output similar mas com diferentes sintaxes
---

# 1. Bibliotecas

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import yfinance as yf

# 2. Obtenção dos dados

In [2]:
df = yf.download('^BVSP', start='2000-01-01', end='2023-01-10')
df['MM_9'] = df.Close.rolling(9).mean()

[*********************100%***********************]  1 of 1 completed


# 3. Construção das funções

Funções que percorrem as linhas do dataframe em busca de operações de trading

## 3.1. Primeira etapa

Função que avalia se cruzamento de preço na média móvel

In [6]:
df

,Open,High,Low,Close,Adj Close,Volume,MM_9
Date,,,,,,,
2000-01-03,17098.0,17408.0,16719.0,16930.0,16930.0,0,NaN
2000-01-04,16908.0,16908.0,15851.0,15851.0,15851.0,0,NaN
2000-01-05,15871.0,16302.0,15350.0,16245.0,16245.0,0,NaN
2000-01-06,16237.0,16499.0,15977.0,16107.0,16107.0,0,NaN
2000-01-07,16125.0,16449.0,16125.0,16309.0,16309.0,0,NaN
...,...,...,...,...,...,...,...
2023-01-03,106377.0,106684.0,103852.0,104166.0,104166.0,14466700,108024.777778
2023-01-04,104167.0,105627.0,103915.0,105334.0,105334.0,14451200,107804.888889
2023-01-05,105336.0,107743.0,105333.0,107518.0,107518.0,15512000,107827.222222


In [5]:
def cruzamento_preco_mm(df):

    cond1_compra = df.Close > df.MM_9
    cond2_compra = df.Close.shift(1) <= df.MM_9.shift(1)

    cond1_encerre = df.Close < df.MM_9
    cond2_encerre = df.Close.shift(1) >= df.MM_9.shift(1)

    if (cond1_compra & cond2_compra):
        
        return 'compra'

    elif (cond1_encerre & cond2_encerre):
        
        return 'encerre'

In [9]:
row in df

NameError: name 'row' is not defined

In [16]:
teste = [cruzamento_preco_mm(df, i) for (df['Close'], df.MM_9) in zip(df['Close'], df.MM_9)]

NameError: name 'i' is not defined

In [4]:
cruzamento_preco_mm(df)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [14]:
def cruzamento_preco_mm(df, i):

    cond1_compra = df.Close[i] > df.MM_9[i]
    cond2_compra = df.Close.shift(1)[i] <= df.MM_9.shift(1)[i]

    cond1_encerre = df.Close[i] < df.MM_9[i]
    cond2_encerre = df.Close.shift(1)[i] >= df.MM_9.shift(1)[i]
    
    if (cond1_compra & cond2_compra):
        return 'compra'

    if (cond1_encerre & cond1_encerre):
        return 'encerre'

In [54]:
def backtest_cruz_preco_mm(df):
    
    position = 0
    dict_operacoes = {'data_entrada':[], 'preco_entrada':[], 'data_saida':[], 'preco_saida':[]}

    for i in range(len(df)):

        if (position==0) and (cruzamento_preco_mm(df, i)=='compra'):
            dict_operacoes['data_entrada'].append(df.index[i])
            dict_operacoes['preco_entrada'].append(df.Close[i])
            position=1

        elif (position==1) and (cruzamento_preco_mm(df, i)=='encerre'):
            dict_operacoes['data_saida'].append(df.index[i])
            dict_operacoes['preco_saida'].append(df.Close[i])
            position=0
    
    return pd.DataFrame.from_dict(dict_operacoes, orient='index').transpose()

In [64]:
%%timeit -r3 -n2
backtest_cruz_preco_mm(df)

1.41 s ± 8.13 ms per loop (mean ± std. dev. of 3 runs, 2 loops each)


In [201]:
df_invertido = df.iloc[::-1]
df_invertido.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-12-29,110237.0,111178.0,109560.0,110031.0,110031.0,13389100
2022-12-28,108578.0,110536.0,108578.0,110237.0,110237.0,11193900
2022-12-27,108739.0,109353.0,107418.0,108347.0,108347.0,11158100
2022-12-26,109699.0,109755.0,108309.0,108738.0,108738.0,5219500
2022-12-23,107552.0,109994.0,107552.0,109698.0,109698.0,12847100


In [202]:
df_invertido = df.iloc[::-1]
df_invertido['Low'] <= df_invertido['Low'].rolling(2, closed='left').min()

Date
2022-12-29    False
2022-12-28    False
2022-12-27     True
2022-12-26    False
2022-12-23    False
              ...  
2022-01-07    False
2022-01-06     True
2022-01-05     True
2022-01-04    False
2022-01-03    False
Name: Low, Length: 250, dtype: bool

In [ ]:
def localizar_fundos(df, n_candles=2):
    
    df_invertido = df.iloc[::-1]

    cond_low_anteriores = df['Low'] <= df['Low'].rolling(n_candles, closed='left').min()
    cond_low_posteriores = df_invertido['Low'] <= df_invertido['Low'].rolling(n_candles, closed='left').min()

    return (cond_low_anteriores & cond_low_posteriores)

In [203]:
localizar_fundos(df)

Date
2022-01-03    False
2022-01-04    False
2022-01-05     True
2022-01-06    False
2022-01-07    False
              ...  
2022-12-23    False
2022-12-26    False
2022-12-27     True
2022-12-28    False
2022-12-29    False
Name: Low, Length: 250, dtype: bool

In [204]:
def localizar_topos(df, n_candles=2):
    
    df_invertido = df.iloc[::-1]

    cond_high_anteriores = df['High'] >= df['High'].rolling(n_candles, closed='left').max()
    cond_high_posteriores = df_invertido['High'] >= df_invertido['High'].rolling(n_candles, closed='left').max()
  
    return (cond_high_anteriores & cond_high_posteriores)

In [206]:
localizar_topos(df, 3)

Date
2022-01-03    False
2022-01-04    False
2022-01-05    False
2022-01-06    False
2022-01-07    False
              ...  
2022-12-23    False
2022-12-26    False
2022-12-27    False
2022-12-28    False
2022-12-29    False
Name: High, Length: 250, dtype: bool

In [207]:
n_candles = 2

dict_topos_fundos = {'vertice':[], 'data':[], 'preco':[]}

for i in range(len(df)):

    if localizar_fundos(df, n_candles).iloc[i]:
        dict_topos_fundos['vertice'].append('fundo')
        dict_topos_fundos['data'].append(df.index[i])
        dict_topos_fundos['preco'].append(df.Low.iloc[i])
    
    if localizar_topos(df, n_candles).iloc[i]:
        dict_topos_fundos['vertice'].append('topo')
        dict_topos_fundos['data'].append(df.index[i])
        dict_topos_fundos['preco'].append(df.High.iloc[i])

In [209]:
df_topos_fundos = pd.DataFrame(dict_topos_fundos)
df_topos_fundos.index = df_topos_fundos.data
df_topos_fundos.head(10)

,vertice,data,preco
data,,,
2022-01-05,fundo,2022-01-05,100850.0
2022-01-14,topo,2022-01-14,107062.0
2022-01-20,topo,2022-01-20,109873.0
2022-01-24,fundo,2022-01-24,106624.0
2022-01-27,topo,2022-01-27,113057.0
2022-01-31,fundo,2022-01-31,111195.0
2022-02-02,topo,2022-02-02,113666.0
2022-02-04,fundo,2022-02-04,110321.0
2022-02-11,topo,2022-02-11,114899.0


In [210]:
df_final = pd.merge(df, df_topos_fundos, left_index=True, right_index=True, how='left')
df_final.head(10)

,Open,High,Low,Close,Adj Close,Volume,vertice,data,preco
Date,,,,,,,,,
2022-01-03,104823.0,106125.0,103413.0,103922.0,103922.0,11128500,NaN,NaT,NaN
2022-01-04,103922.0,104276.0,103096.0,103514.0,103514.0,11491600,NaN,NaT,NaN
2022-01-05,103514.0,103514.0,100850.0,101006.0,101006.0,0,fundo,2022-01-05,100850.0
2022-01-06,101006.0,102235.0,101000.0,101561.0,101561.0,11749200,NaN,NaT,NaN
2022-01-07,101561.0,102719.0,101104.0,102719.0,102719.0,11733200,NaN,NaT,NaN
2022-01-10,102719.0,102719.0,101038.0,101945.0,101945.0,10264700,NaN,NaT,NaN
2022-01-11,101946.0,103780.0,101918.0,103779.0,103779.0,12421500,NaN,NaT,NaN
2022-01-12,103779.0,105869.0,103771.0,105686.0,105686.0,13026500,NaN,NaT,NaN
2022-01-13,105686.0,106251.0,104974.0,105530.0,105530.0,11712300,NaN,NaT,NaN


In [211]:
df_figure = df_final

fig_topos_fundos = go.Figure(data=[go.Candlestick(name='', x=df_figure.index, open=df_figure.Open, high = df_figure.High,
                                                  low=df_figure.Low, close=df_figure.Close, 
                                                  increasing_line_color= 'green', decreasing_line_color= 'red',
                                                  showlegend=False)])

fig_topos_fundos.add_trace(go.Scatter(x=df_topos_fundos.data, y=df_topos_fundos.preco,
                                      mode="lines", line_width=2, line_dash="solid", line_color="black",
                                      name="Topos / Fundos", showlegend=True))

fig_topos_fundos.update_xaxes(rangebreaks=[
                              dict(bounds=["sat", "mon"])])

fig_topos_fundos.update_layout(xaxis_rangeslider_visible=False, title_text='<b>Indicador topos e fundos',
                               template = 'none', margin=dict(l=75, r=75, t=75, b=75), 
                               paper_bgcolor="#f7f8fa",
                               width=1200,height=600)

## 3.2. Segunda etapa

In [212]:
n_candles = 2
ls_vertices = ['neutro']

dict_topos_fundos = {'vertice':[], 'data':[], 'preco':[]}

for i in range(len(df)):

    if localizar_fundos(df, n_candles).iloc[i]:

        if ls_vertices[-1]!='fundo':
            dict_topos_fundos['vertice'].append('fundo')
            dict_topos_fundos['data'].append(df.index[i])
            dict_topos_fundos['preco'].append(df.Low.iloc[i])
            ls_vertices.append('fundo')
        
        else:
            if df.Low.iloc[i] < dict_topos_fundos['preco'][-1]:
                dict_topos_fundos['data'][-1] = df.index[i]
                dict_topos_fundos['preco'][-1] = df.Low.iloc[i]
    
    if localizar_topos(df, n_candles).iloc[i]:

        if ls_vertices[-1]!='topo':
            dict_topos_fundos['vertice'].append('topo')
            dict_topos_fundos['data'].append(df.index[i])
            dict_topos_fundos['preco'].append(df.High.iloc[i])
            ls_vertices.append('topo')

        else:
            if df.High.iloc[i] > dict_topos_fundos['preco'][-1]:
                dict_topos_fundos['data'][-1] = df.index[i]
                dict_topos_fundos['preco'][-1] = df.High.iloc[i]

In [213]:
df_topos_fundos = pd.DataFrame(dict_topos_fundos)
df_topos_fundos.index = df_topos_fundos.data

df_final = pd.merge(df, df_topos_fundos, left_index=True, right_index=True, how='left')
df_final.head(20)

df_figure = df_final

fig_topos_fundos = go.Figure(data=[go.Candlestick(name='', x=df_figure.index, open=df_figure.Open, high = df_figure.High,
                                                  low=df_figure.Low, close=df_figure.Close, 
                                                  increasing_line_color= 'green', decreasing_line_color= 'red',
                                                  showlegend=False)])

fig_topos_fundos.add_trace(go.Scatter(x=df_topos_fundos.data, y=df_topos_fundos.preco,
                                      mode="lines", line_width=2, line_dash="solid", line_color="black",
                                      name="Topos / Fundos", showlegend=True))

fig_topos_fundos.update_xaxes(rangebreaks=[
                              dict(bounds=["sat", "mon"])])

fig_topos_fundos.update_layout(xaxis_rangeslider_visible=False, title_text='<b>Indicador topos e fundos',
                               template = 'none', margin=dict(l=75, r=75, t=75, b=75), 
                               paper_bgcolor="#f7f8fa",
                               width=1200,height=600)

In [214]:
n_candles = 10
ls_vertices = ['neutro']

dict_topos_fundos = {'vertice':[], 'data':[], 'preco':[]}

for i in range(len(df)):

    if localizar_fundos(df, n_candles).iloc[i]:

        if ls_vertices[-1]!='fundo':
            dict_topos_fundos['vertice'].append('fundo')
            dict_topos_fundos['data'].append(df.index[i])
            dict_topos_fundos['preco'].append(df.Low.iloc[i])
            ls_vertices.append('fundo')
        
        else:
            if df.Low.iloc[i] < dict_topos_fundos['preco'][-1]:
                dict_topos_fundos['data'][-1] = df.index[i]
                dict_topos_fundos['preco'][-1] = df.Low.iloc[i]
    
    if localizar_topos(df, n_candles).iloc[i]:

        if ls_vertices[-1]!='topo':
            dict_topos_fundos['vertice'].append('topo')
            dict_topos_fundos['data'].append(df.index[i])
            dict_topos_fundos['preco'].append(df.High.iloc[i])
            ls_vertices.append('topo')

        else:
            if df.High.iloc[i] > dict_topos_fundos['preco'][-1]:
                dict_topos_fundos['data'][-1] = df.index[i]
                dict_topos_fundos['preco'][-1] = df.High.iloc[i]

df_topos_fundos = pd.DataFrame(dict_topos_fundos)
df_topos_fundos.index = df_topos_fundos.data

df_final = pd.merge(df, df_topos_fundos, left_index=True, right_index=True, how='left')
df_final.head(20)

df_figure = df_final

fig_topos_fundos = go.Figure(data=[go.Candlestick(name='', x=df_figure.index, open=df_figure.Open, high = df_figure.High,
                                                  low=df_figure.Low, close=df_figure.Close, 
                                                  increasing_line_color= 'green', decreasing_line_color= 'red',
                                                  showlegend=False)])

fig_topos_fundos.add_trace(go.Scatter(x=df_topos_fundos.data, y=df_topos_fundos.preco,
                                      mode="lines", line_width=2, line_dash="solid", line_color="black",
                                      name="Topos / Fundos", showlegend=True))

fig_topos_fundos.update_xaxes(rangebreaks=[
                              dict(bounds=["sat", "mon"])])

fig_topos_fundos.update_layout(xaxis_rangeslider_visible=False, title_text='<b>Indicador topos e fundos',
                               template = 'none', margin=dict(l=75, r=75, t=75, b=75), 
                               paper_bgcolor="#f7f8fa",
                               width=1200,height=600)

# 4. Teoria de Dow

In [215]:
df_1w = yf.download('^BVSP', start='2020-01-01', end='2023-01-01', interval='1wk')
df_1d = yf.download('^BVSP', start='2020-01-01', end='2023-01-01', interval='1d')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [216]:
df = df_1w.copy()

n_candles = 10
ls_vertices = ['neutro']

dict_topos_fundos = {'vertice':[], 'data':[], 'preco':[]}

for i in range(len(df)):

    if localizar_fundos(df, n_candles).iloc[i]:

        if ls_vertices[-1]!='fundo':
            dict_topos_fundos['vertice'].append('fundo')
            dict_topos_fundos['data'].append(df.index[i])
            dict_topos_fundos['preco'].append(df.Low.iloc[i])
            ls_vertices.append('fundo')
        
        else:
            if df.Low.iloc[i] < dict_topos_fundos['preco'][-1]:
                dict_topos_fundos['data'][-1] = df.index[i]
                dict_topos_fundos['preco'][-1] = df.Low.iloc[i]
    
    if localizar_topos(df, n_candles).iloc[i]:

        if ls_vertices[-1]!='topo':
            dict_topos_fundos['vertice'].append('topo')
            dict_topos_fundos['data'].append(df.index[i])
            dict_topos_fundos['preco'].append(df.High.iloc[i])
            ls_vertices.append('topo')

        else:
            if df.High.iloc[i] > dict_topos_fundos['preco'][-1]:
                dict_topos_fundos['data'][-1] = df.index[i]
                dict_topos_fundos['preco'][-1] = df.High.iloc[i]

df_topos_fundos = pd.DataFrame(dict_topos_fundos)
df_topos_fundos.index = df_topos_fundos.data

df_final = pd.merge(df, df_topos_fundos, left_index=True, right_index=True, how='left')
df_final.head(20)

df_figure = df_final

fig_topos_fundos = go.Figure(data=[go.Candlestick(name='', x=df_figure.index, open=df_figure.Open, high = df_figure.High,
                                                  low=df_figure.Low, close=df_figure.Close, 
                                                  increasing_line_color= 'green', decreasing_line_color= 'red',
                                                  showlegend=False)])

fig_topos_fundos.add_trace(go.Scatter(x=df_topos_fundos.data, y=df_topos_fundos.preco,
                                      mode="lines", line_width=2, line_dash="solid", line_color="black",
                                      name="Topos / Fundos", showlegend=True))

fig_semanal = fig_topos_fundos.update_layout(xaxis_rangeslider_visible=False, title_text='<b>Indicador topos e fundos',
                               template = 'none', margin=dict(l=75, r=75, t=75, b=75), 
                               paper_bgcolor="#f7f8fa",
                               width=1200,height=600)

In [217]:
df = df_1d.copy()

n_candles = 10
ls_vertices = ['neutro']

dict_topos_fundos = {'vertice':[], 'data':[], 'preco':[]}

for i in range(len(df)):

    if localizar_fundos(df, n_candles).iloc[i]:

        if ls_vertices[-1]!='fundo':
            dict_topos_fundos['vertice'].append('fundo')
            dict_topos_fundos['data'].append(df.index[i])
            dict_topos_fundos['preco'].append(df.Low.iloc[i])
            ls_vertices.append('fundo')
        
        else:
            if df.Low.iloc[i] < dict_topos_fundos['preco'][-1]:
                dict_topos_fundos['data'][-1] = df.index[i]
                dict_topos_fundos['preco'][-1] = df.Low.iloc[i]
    
    if localizar_topos(df, n_candles).iloc[i]:

        if ls_vertices[-1]!='topo':
            dict_topos_fundos['vertice'].append('topo')
            dict_topos_fundos['data'].append(df.index[i])
            dict_topos_fundos['preco'].append(df.High.iloc[i])
            ls_vertices.append('topo')

        else:
            if df.High.iloc[i] > dict_topos_fundos['preco'][-1]:
                dict_topos_fundos['data'][-1] = df.index[i]
                dict_topos_fundos['preco'][-1] = df.High.iloc[i]

df_topos_fundos = pd.DataFrame(dict_topos_fundos)
df_topos_fundos.index = df_topos_fundos.data

df_final = pd.merge(df, df_topos_fundos, left_index=True, right_index=True, how='left')
df_final.head(20)

df_figure = df_final

fig_topos_fundos = go.Figure(data=[go.Candlestick(name='', x=df_figure.index, open=df_figure.Open, high = df_figure.High,
                                                  low=df_figure.Low, close=df_figure.Close, 
                                                  increasing_line_color= 'green', decreasing_line_color= 'red',
                                                  showlegend=False)])

fig_topos_fundos.add_trace(go.Scatter(x=df_topos_fundos.data, y=df_topos_fundos.preco,
                                      mode="lines", line_width=2, line_dash="solid", line_color="black",
                                      name="Topos / Fundos", showlegend=True))

fig_diario = fig_topos_fundos.update_layout(xaxis_rangeslider_visible=False, title_text='<b>Indicador topos e fundos',
                               template = 'none', margin=dict(l=75, r=75, t=75, b=75), 
                               paper_bgcolor="#f7f8fa",
                               width=1200,height=600)

In [218]:
fig_semanal

In [219]:
fig_diario

# 5. Screening ativos

In [220]:
lista_tickers = ['ABEV3', 'ALPA4', 'ALSO3', 'ARZZ3', 'ASAI3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4',
                'BBSE3', 'BEEF3', 'BPAC11', 'BPAN4', 'BRAP4', 'BRFS3', 'BRKM5', 'CASH3', 'CCRO3', 'CIEL3', 'CMIG4', 'CMIN3',
                'COGN3', 'CPFE3', 'CPLE6', 'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3', 'CYRE3', 'DXCO3', 'ECOR3', 'EGIE3', 'ELET3',
                'ELET6', 'EMBR3', 'ENBR3', 'ENEV3', 'ENGI11', 'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4', 'GOAU4', 'GOLL4', 'HAPV3',
                'HYPE3', 'IGTI11', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'LREN3', 'LWSA3', 'MGLU3', 'MRFG3', 'MRVE3', 'MULT3',
                'NTCO3', 'PCAR3', 'PETR3', 'PETR4', 'PETZ3', 'PRIO3', 'QUAL3', 'RADL3', 'RAIL3', 'RAIZ4', 'RDOR3', 'RENT3',
                'RRRP3', 'SANB11', 'SBSP3', 'SLCE3', 'SMTO3', 'SOMA3', 'SUZB3', 'TAEE11', 'TIMS3', 'TOTS3', 'UGPA3', 'USIM5',
                'VALE3', 'VBBR3', 'VIIA3', 'VIVT3', 'WEGE3', 'YDUQ3']

In [222]:
lista_tickers_yf = [x + '.SA' for x in lista_tickers]

In [223]:
lista_tickers_yf[0:5]

['ABEV3.SA', 'ALPA4.SA', 'ALSO3.SA', 'ARZZ3.SA', 'ASAI3.SA']

In [224]:
ls_close_ativos = []
indice_ativo = 0

for i in lista_tickers_yf:
    df_1d_ativo = yf.download(i, start='2022-01-01', interval='1d', auto_adjust=True)
    df_1d_ativo['Ticker'] = i
    df_1d_ativo['Indice_ativo'] = indice_ativo
    indice_ativo +=1
    ls_close_ativos.append(df_1d_ativo)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [225]:
list(enumerate(lista_tickers_yf))

[(0, 'ABEV3.SA'),
 (1, 'ALPA4.SA'),
 (2, 'ALSO3.SA'),
 (3, 'ARZZ3.SA'),
 (4, 'ASAI3.SA'),
 (5, 'AZUL4.SA'),
 (6, 'B3SA3.SA'),
 (7, 'BBAS3.SA'),
 (8, 'BBDC3.SA'),
 (9, 'BBDC4.SA'),
 (10, 'BBSE3.SA'),
 (11, 'BEEF3.SA'),
 (12, 'BPAC11.SA'),
 (13, 'BPAN4.SA'),
 (14, 'BRAP4.SA'),
 (15, 'BRFS3.SA'),
 (16, 'BRKM5.SA'),
 (17, 'CASH3.SA'),
 (18, 'CCRO3.SA'),
 (19, 'CIEL3.SA'),
 (20, 'CMIG4.SA'),
 (21, 'CMIN3.SA'),
 (22, 'COGN3.SA'),
 (23, 'CPFE3.SA'),
 (24, 'CPLE6.SA'),
 (25, 'CRFB3.SA'),
 (26, 'CSAN3.SA'),
 (27, 'CSNA3.SA'),
 (28, 'CVCB3.SA'),
 (29, 'CYRE3.SA'),
 (30, 'DXCO3.SA'),
 (31, 'ECOR3.SA'),
 (32, 'EGIE3.SA'),
 (33, 'ELET3.SA'),
 (34, 'ELET6.SA'),
 (35, 'EMBR3.SA'),
 (36, 'ENBR3.SA'),
 (37, 'ENEV3.SA'),
 (38, 'ENGI11.SA'),
 (39, 'EQTL3.SA'),
 (40, 'EZTC3.SA'),
 (41, 'FLRY3.SA'),
 (42, 'GGBR4.SA'),
 (43, 'GOAU4.SA'),
 (44, 'GOLL4.SA'),
 (45, 'HAPV3.SA'),
 (46, 'HYPE3.SA'),
 (47, 'IGTI11.SA'),
 (48, 'ITSA4.SA'),
 (49, 'ITUB4.SA'),
 (50, 'JBSS3.SA'),
 (51, 'KLBN11.SA'),
 (52, 'LREN3.SA'),

In [226]:
ls_close_ativos[48]

,Open,High,Low,Close,Volume,Ticker,Indice_ativo
Date,,,,,,,
2022-01-03,7.538162,7.680233,7.454589,7.538162,31396860.0,ITSA4.SA,48
2022-01-04,7.546517,7.663517,7.496374,7.596660,42714320.0,ITSA4.SA,48
2022-01-05,7.563232,7.613376,7.454589,7.513089,38184080.0,ITSA4.SA,48
2022-01-06,7.504733,7.638446,7.471303,7.579947,27922180.0,ITSA4.SA,48
2022-01-07,7.579947,7.680233,7.529805,7.671876,23590930.0,ITSA4.SA,48
...,...,...,...,...,...,...,...
2023-05-02,8.670000,8.700000,8.310000,8.380000,31006000.0,ITSA4.SA,48
2023-05-03,8.370000,8.420000,8.330000,8.370000,15887900.0,ITSA4.SA,48
2023-05-04,8.410000,8.540000,8.320000,8.480000,26512500.0,ITSA4.SA,48


In [227]:
n_candles = 10

ls_ativos_topos_fundos = []

for df in ls_close_ativos:
    
    ls_vertices = ['neutro']
    dict_topos_fundos = {'vertice':[], 'data':[], 'preco':[]}

    for i in range(len(df)):

        if localizar_fundos(df, n_candles).iloc[i]:

            if ls_vertices[-1]!='fundo':
                dict_topos_fundos['vertice'].append('fundo')
                dict_topos_fundos['data'].append(df.index[i])
                dict_topos_fundos['preco'].append(df.Low.iloc[i])
                ls_vertices.append('fundo')
            
            else:
                if df.Low.iloc[i] < dict_topos_fundos['preco'][-1]:
                    dict_topos_fundos['data'][-1] = df.index[i]
                    dict_topos_fundos['preco'][-1] = df.Low.iloc[i]
        
        if localizar_topos(df, n_candles).iloc[i]:

            if ls_vertices[-1]!='topo':
                dict_topos_fundos['vertice'].append('topo')
                dict_topos_fundos['data'].append(df.index[i])
                dict_topos_fundos['preco'].append(df.High.iloc[i])
                ls_vertices.append('topo')

            else:
                if df.High.iloc[i] > dict_topos_fundos['preco'][-1]:
                    dict_topos_fundos['data'][-1] = df.index[i]
                    dict_topos_fundos['preco'][-1] = df.High.iloc[i]
    
    df_topos_fundos = pd.DataFrame(dict_topos_fundos)
    df_topos_fundos.index = df_topos_fundos.data

    df_final = pd.merge(df, df_topos_fundos, left_index=True, right_index=True, how='left')
    df_final.head(20)

    ls_ativos_topos_fundos.append(df_final)

In [228]:
def ativo_rompeu_topo(df):
    ultimo_topo = df[df.vertice=='topo']['preco'][-1]
    distancia_topo = (ultimo_topo - df.Close[-1])/ultimo_topo*100
    
    if distancia_topo < 0:
        return 'rompeu topo'

In [229]:
for df in ls_ativos_topos_fundos:

    if ativo_rompeu_topo(df)=='rompeu topo':
        print(df.Ticker[0],'indice=',df.Indice_ativo[0],'rompeu topo anterior')

ALSO3.SA indice= 2 rompeu topo anterior
B3SA3.SA indice= 6 rompeu topo anterior
BBAS3.SA indice= 7 rompeu topo anterior
BBDC3.SA indice= 8 rompeu topo anterior
BBDC4.SA indice= 9 rompeu topo anterior
BPAC11.SA indice= 12 rompeu topo anterior
BRFS3.SA indice= 15 rompeu topo anterior
BRKM5.SA indice= 16 rompeu topo anterior
CCRO3.SA indice= 18 rompeu topo anterior
CIEL3.SA indice= 19 rompeu topo anterior
CMIG4.SA indice= 20 rompeu topo anterior
COGN3.SA indice= 22 rompeu topo anterior
CPFE3.SA indice= 23 rompeu topo anterior
CYRE3.SA indice= 29 rompeu topo anterior
DXCO3.SA indice= 30 rompeu topo anterior
ECOR3.SA indice= 31 rompeu topo anterior
EZTC3.SA indice= 40 rompeu topo anterior
FLRY3.SA indice= 41 rompeu topo anterior
IGTI11.SA indice= 47 rompeu topo anterior
ITSA4.SA indice= 48 rompeu topo anterior
ITUB4.SA indice= 49 rompeu topo anterior
KLBN11.SA indice= 51 rompeu topo anterior
LWSA3.SA indice= 53 rompeu topo anterior
MGLU3.SA indice= 54 rompeu topo anterior
MRVE3.SA indice= 5

In [232]:
i = 6

df_figure = ls_ativos_topos_fundos[i]
df_topos_fundos = ls_ativos_topos_fundos[i][~ls_ativos_topos_fundos[i].preco.isna()]

fig_topos_fundos = go.Figure(data=[go.Candlestick(name='', x=df_figure.index, open=df_figure.Open, high = df_figure.High,
                                                  low=df_figure.Low, close=df_figure.Close, 
                                                  increasing_line_color= 'green', decreasing_line_color= 'red',
                                                  showlegend=False)])

fig_topos_fundos.add_trace(go.Scatter(x=df_topos_fundos.index, y=df_topos_fundos.preco,
                                      mode="lines", line_width=2, line_dash="solid", line_color="black",
                                      name="Topos / Fundos", showlegend=True))

fig_topos_fundos.update_layout(xaxis_rangeslider_visible=False, title_text='<b>Indicador topos e fundos',
                               template = 'none', margin=dict(l=75, r=75, t=75, b=75), 
                               paper_bgcolor="#f7f8fa",
                               width=1200,height=600)